In [7]:
import os, time, threading
from google.colab import output


DELTA_URL = "https://delta.xanmarta.workers.dev/0:/"


def initiation():
    delta_folder = "/content/delta-worker"
    work_list = ['DELTA.ipynb', 'MEGA.ipynb']
    for work in work_list:
        print(work)
        work_path = os.path.join(delta_folder, work)
        %run {work_path}
    if os.path.isdir('/content/sample_data'):
        !rm -r '/content/sample_data'


def download(filePath):
    # Download to '/content/file'
    full_url = os.path.join(DELTA_URL, filePath)
    fileName = os.path.basename(filePath)
    full_path = '/content/' + fileName
    print('Downloading', full_url)
    trying = 0
    while True:
        if os.path.isfile(full_path):
            !rm '$full_path'
        !curl '$full_url' -o '$full_path'
        if not os.path.isfile(full_path):
            print('File not found on host:', full_url)
            return False
        with open(full_path, 'r') as f:
            content = f.read()
        if (not '<!DOCTYPE html>' in content) and (not 'usageLimits' in content):
            return True
        trying += 1
        if trying == 10:
            print('Download failed')
            return False
        print('Download failed. Trying again')
        time.sleep(1)


def clear_output():
    while True:
        output.clear()
        time.sleep(200)


class BaseWorker:
    def __init__(self, name, state):
        self.files = []
        self.path = os.path.join(name, state)
        self.success = True

    def prepare(self):
        for nf in self.files:
            success = download(os.path.join(self.path, nf))
            if not success:
                return False
        return True

    def working(self):
        print('Nothing to work')


# Main worker
def worker(name, state):
    print('cp1')
    initiation()
    print('cp2')
    if download(os.path.join(name, state, 'work.txt')):
        with open('/content/work.txt', 'r') as f:
            work = f.read().strip('\n')
            if work == 'DELTA':
                runner = DeltaWorker(name, state)
            elif work == 'MEGA':
                runner = MegaWorker(name, state)
            else:
                print('No work to do')
                return
        threading.Thread(target=clear_output).start()
        if runner.prepare():
            print('cp3')
            runner.working()
            print('cp4')
            if runner.success:
                print('----- Successfully -----')
            else:
                print('----- Unsuccessfully -----')